# Dye tracking

This notebook estimates surface azimuthal velocity from laboratory videos using the dye-tracking method described in Section 3.1 of David et al. (2024) (https://doi.org/10.1017/jfm.2024.674).

In [ ]:
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt

from matplotlib import font_manager
import matplotlib

import skimage
from skimage import feature
from skimage.color import rgb2gray
from skimage import color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter
from skimage import morphology
from skimage import transform

# $ sudo pip install scikit-video
import skvideo

import skvideo.io

import cv2

from scipy import stats
from scipy import special

import pandas as pd
import glob
import os
import re

# $ pip install localreg
from localreg import *

# $ pip install findiff
import findiff

# $ pip install sktime
from sktime.transformations.series.outlier_detection import HampelFilter
from sktime.transformations.series.impute import Imputer

## Define processing and analysis functions

In [ ]:
def make_polar_meshgrids(videodata,nr,chi_channel,r_o_meters,warp=True,α=1.25,m=2.5,plot=False,cannysigma=7,cannylowthresh=10,cannyhighthresh=50,dcxi=0,dcyi=0,dcxo=0,dcyo=0,contract=0.99,expand=1.05,figuresize=(10, 5),plotzoom=1):
    '''
    Parameters
    ----------
    videodata : 4D array
        RGB color video of experiment, cropped so that the outer channel diameter is fills at least 80% of the frame's width
    nr : int
        Number of radial gridpoints.
    chi_channel : float
        Ratio of outer to inner radii of channel (known from physical measurements)
    r_o_meters : float
        Outer radius of annular channel in meters
    warp : boolean, optional
        If True, finds the homography mapping the original image to the perspective-warped image.
    α : float, optional
        Shape parameter for non-uniform grid. Mapping converts uniform grid on [0,1] to non-uniform grid on [r_l,r_u] according
        to r_u*((1 + r_l/r_u)/2 + (1 - r_l/r_u)/2*special.erf(m*(np.linspace(0,1,nr)**α - 1/2))). α shifts the
        grid-point density distribution towards the inner boundary for α > 1 and towards the outer boundary for α < 1.
    m : float, optional
        Shape parameter for non-uniform grid. Mapping converts uniform grid on [0,1] to non-uniform grid on [r_l,r_u] according
        to r_u*((1 + r_l/r_u)/2 + (1 - r_l/r_u)/2*special.erf(m*(np.linspace(0,1,nr)**α - 1/2))). The density of gridpoints near
        the boundaries increases with m.
    plot : boolean, optional
        If True, plots channel edges found by Hough transform as well as the chosen annular domain boundaries
    cannysigma : float, optional
        Standard deviation of the Gaussian filter for Canny edge detection of channel boundaries.
    cannylowthresh : float, optional
        Lower bound for hysteresis thresholding (linking edges) for Canny edge detection of channel boundaries.
    cannyhighthresh : float, optional
        Upper bound for hysteresis thresholding (linking edges) for Canny edge detection of channel boundaries
    dcxi : float, optional
        Increment by which to manually adjust the horizontal (x) position of inner circle center
    dcyi : float, optional
        Increment by which to manually adjust the vertical (y) position of inner circle center
    dcxo : float, optional
        Increment by which to manually adjust the horizontal (x) position of outer circle center
    dcyo : float, optional
        Increment by which to manually adjust the vertical (y) position of outer circle center
    contract : float, optional
        Fraction by which to contract the outer edge of annular domain
    expand : float, optional
        Fraction by which to expand inner edge into the annular domain
    figuresize : tuple, optional
        Dimensions of optional plot
    plotzoom : float, optional
        Factor by which to enlarge the plot
        
    Returns
    -------
    homography : 2D array
        Homography mapping the original image to the perspective-warped image.
    annular_params: list
        Annular domain parameters [cx_o,cy_o,r_u,r_l,r_i,r_o]:  (x) column-index of outer cylinder center, 
        (y) row-index of outer cylinder center, outer radius of annular domain, inner radius of annular domain
        inner radius of annular channel, outer radius of annular channel.
    scale_factor : float
        Scaling from pixels to physical units of length (meters). Units: meters/pixel.
    rgarr : 1D array
        Array of radial gridpoints. dr = (r_u - r_l)/nr; rgarr = np.array([r_l + (1/2+n)*dr for n in range(nr)]).
    rmg_index : int
        Index of the point in rgarr that corresponds to the mid-gap
    rmaskstack : 3D array
        Stack of circle masks for each radius in rgarr
    thetagrid : 2D array
        "True" angular position values on [0,2π). Theta=0 for x=0+ on the -y-axis and Theta=2π for x=0- on the -y-axis.
    theta2grid : 2D array
        Shifted angular position values on [-π,π). Theta=-π for x=0- on the +y-axis and Theta=π for x=0+ on the +y-axis.
    theta3grid : 2D array
        Shifted angular position values on [-π/2,3π/2). Theta=-π/2 for y=0- on the -x-axis and Theta=3π/2 for y=0+ on the -x-axis.
    theta4grid : 2D array
        Shifted angular position values on [π/2,5π/2). Theta=π/2 for y=0- on the +x-axis and Theta=5π/2 for y=0+ on the +x-axis.
    '''
    
    im = videodata[0]/255.
    gray0 =rgb2gray(im)
    im8bit = skimage.img_as_ubyte(gray0)
    alledges = feature.canny(im8bit, sigma=cannysigma,low_threshold=cannylowthresh, high_threshold=cannyhighthresh)
    
    # Channel detection
    ## Detect two radii
    hough_radii_outer = np.arange(0.8*np.min(im8bit.shape)/2, 1.2*np.min(im8bit.shape)/2,10).astype('int')
    chi = chi_channel # adjust for channel
    hough_radii_inner = np.arange(0.8*chi*np.min(im8bit.shape)/2, 1.2*chi*np.min(im8bit.shape)/2,10).astype('int')
    hough_res_outer = hough_circle(alledges, hough_radii_outer)
    hough_res_inner = hough_circle(alledges, hough_radii_inner)
    ## Select the most prominent circle
    accums_outer, cx_o_arr, cy_o_arr, r_o_arr = hough_circle_peaks(hough_res_outer, hough_radii_outer,
                                               total_num_peaks=1)
    accums_inner, cx_i_arr, cy_i_arr, r_i_arr = hough_circle_peaks(hough_res_inner, hough_radii_inner,
                                               total_num_peaks=1)
    [cx_o,cy_o,r_o] = [i[0] for i in [cx_o_arr, cy_o_arr, r_o_arr]]
    [cx_i,cy_i,r_i] = [i[0] for i in [cx_i_arr, cy_i_arr, r_i_arr]]
    # Manually adjust inner and outer circle centers
    cx_i = cx_i + dcxi
    cy_i = cy_i + dcyi
    cx_o = cx_o + dcxo
    cy_o = cy_o + dcyo
    
    if warp == True:
        # Perspective warp 
        ## Choose source and destination points for homography inversion. Pts on outer circle will map to themselves; pts on inner circle will map to a circle with the same radius but concentric w/ outer circle.
        srcptangles = np.arange(0,2*np.pi,2*np.pi/8)
        pts_srcinner = [[cx_i+r_i*np.cos(srcptangles)[i],cy_i+r_i*np.sin(srcptangles)[i]] for i in np.arange(len(srcptangles))]
        pts_srcouter = [[cx_o+r_o*np.cos(srcptangles)[i],cy_o+r_o*np.sin(srcptangles)[i]] for i in np.arange(len(srcptangles))]
        pts_src = np.array(pts_srcinner+pts_srcouter+[[cx_i,cy_i]])
        pts_dstinner = [[cx_o+r_i*np.cos(srcptangles)[i],cy_o+r_i*np.sin(srcptangles)[i]] for i in np.arange(len(srcptangles))]
        pts_dstouter = pts_srcouter
        pts_dst = np.array(pts_dstinner+pts_dstouter+[[cx_o,cy_o]])  
        ## Find Homography
        homography, status = cv2.findHomography(pts_src, pts_dst)
        ## Warp source image to destination based on homography
        im_warp = cv2.warpPerspective(im, homography, (im.shape[1],im.shape[0]))
    else:
        homography = np.array([[ 1.,  0., 0.],
                            [0.,  1.,  0.],
                            [0.,  0.,  1.]])
        im_warp = im
    
    # Compute scale factor
    scale_factor = r_o_meters/r_o #meters per pixel
    
    # Define annular domain   
    ## Shrink domain
    r_u = contract*r_o
    r_l = expand*r_i 
    ## Collect annular domain parameters
    annular_params = [cx_o,cy_o,r_u,r_l,r_i,r_o]

    # Map from uniform grid on [0,1] to non-uniform grid on [r_l,r_u]    
    ## Compute non-uniform grid
    rgarr = r_u*1/2*(1 + r_l/r_u - ((-1 + r_l/r_u)*special.erf(m*(-(1/2) + np.linspace(0,1,nr)**α)))/special.erf(m/2))  
    ## Add mid-gap point
    rmg_meters = (r_i_meters+r_o_meters)/2
    rmg_pix = rmg_meters/scale_factor
    rgarr = np.sort(np.concatenate((rgarr,[rmg_pix])))
    rmg_index = np.argmin(np.abs(rgarr-rmg_pix))
    ## Choose half-width of the sampling window about each r
    sigmar = np.min([1,np.min(np.diff(rgarr))])
    
    # Plot
    if plot==True:
        fig, axs = plt.subplots(2,2,figsize=figuresize)

        axs[0,0].imshow(alledges,cmap='binary')

        axs[0,1].imshow(im)
        circpatcho = plt.Circle((cx_o,cy_o),r_o,color='b',fill=False,linewidth=2/60*figuresize[0])
        axs[0,1].add_patch(circpatcho)
        axs[0,1].add_patch(circpatcho)
        circpatchi = plt.Circle((cx_i,cy_i),r_i,color='r',fill=False,linewidth=2/60*figuresize[0])
        axs[0,1].add_patch(circpatchi)
        axs[0,1].scatter(cx_i,cy_i,color='r',s=20)
        axs[0,1].scatter(cx_o,cy_o,color='b',s=20)

        if warp == True:
            axs[1,0].imshow(0.5*rgb2gray(im)+0.5*rgb2gray(im_warp),cmap='binary_r')
            circpatcho = plt.Circle((cx_o,cy_o),r_o,color='b',fill=False,linewidth=2/60*figuresize[0])
            axs[1,0].add_patch(circpatcho)
            axs[1,0].add_patch(circpatcho)
            circpatchi = plt.Circle((cx_i,cy_i),r_i,color='r',fill=False,linewidth=2/60*figuresize[0])
            axs[1,0].add_patch(circpatchi)
            axs[1,0].scatter(*zip(*pts_src),color='r',s=10)
            axs[1,0].scatter(*zip(*pts_dst),color='green',s=10)
            for i in np.arange(len(pts_src)):
                arrowlen = (((pts_dst - pts_src)[i][0])**2 + ((pts_dst - pts_src)[i][1])**2)**(1/2)
                axs[1,0].arrow(pts_src[i][0],pts_src[i][1],(pts_dst - pts_src)[i][0],(pts_dst - pts_src)[i][1],zorder=10,color='orange',
                               length_includes_head=True,head_width=10+0.05*arrowlen**(1/2),head_length=0.5*(10+0.05*arrowlen**(1/2)))

            circpatchw = plt.Circle((cx_o,cy_o),r_i,color='green',fill=False,linewidth=2/60*figuresize[0])
            axs[1,0].add_patch(circpatchw)
            axs[1,0].set_xlim(0.5*np.shape(im)[1]- 1/plotzoom*0.5*np.shape(im)[1], 0.5*np.shape(im)[1]+ 1/plotzoom*0.5*np.shape(im)[1])
            axs[1,0].set_ylim(0.5*np.shape(im)[0]+ 1/plotzoom*0.5*np.shape(im)[0],0.5*np.shape(im)[0]- 1/plotzoom*0.5*np.shape(im)[0])

            axs[1,1].imshow(im_warp)
            circpatcho = plt.Circle((cx_o,cy_o),r_o,color='b',fill=False,linewidth=2/60*figuresize[0])
            axs[1,1].add_patch(circpatcho)
            axs[1,1].add_patch(circpatcho)
            circpatchw = plt.Circle((cx_o,cy_o),r_i,color='green',fill=False,linewidth=2/60*figuresize[0])
            axs[1,1].add_patch(circpatchw)
            circ_u=plt.Circle((cx_o,cy_o),r_u,color='y',fill=False,linewidth=2/60*figuresize[0])
            axs[1,1].add_patch(circ_u)
            circ_l=plt.Circle((cx_o,cy_o),r_l,color='y',fill=False,linewidth=2/60*figuresize[0])
            axs[1,1].add_patch(circ_l)
            axs[1,1].scatter(cx_o,cy_o,color='green',s=20)
            axs[1,1].scatter(cx_o,cy_o,color='b',s=20)


    # Make Cartesian meshgrids and convert to polar
    [imyy,imxx]=np.mgrid[0:np.shape(im)[0],0:np.shape(im)[1]]
    rrgrid = ((imxx-cx_o)**2 + (imyy-cy_o)**2)**(1/2)
    thetagrid = np.arctan2(-(imxx-cx_o),-(imyy-cy_o)) + np.pi
    theta2grid = np.arctan2((imxx-cx_o),(imyy-cy_o))
    theta3grid = np.arctan2(-(imyy-cy_o),(imxx-cx_o)) + np.pi/2
    theta4grid = np.arctan2((imyy-cy_o),-(imxx-cx_o)) + np.pi*3/2

    # Refine
    theta2grid = np.where(np.abs(theta2grid-thetagrid)<np.pi, thetagrid, theta2grid)
    theta3grid = np.where(np.abs(theta3grid-thetagrid)<np.pi, thetagrid, theta3grid)
    theta4grid = np.where(np.abs(theta4grid-thetagrid)<np.pi, thetagrid, theta4grid)

    # Make stack of radial boolean masks
    rmaskstack = np.zeros((np.shape(im)[0],np.shape(im)[1],len(rgarr)))
    for n in np.arange(len(rgarr)):
        rnmask = (rrgrid>=(rgarr[n]-sigmar))&(rrgrid<(rgarr[n]+sigmar))
        rmaskstack[:,:,n]=rnmask
    rmaskstack = rmaskstack.astype('bool')
    
    return homography,annular_params,scale_factor,rgarr,rmg_index,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid

In [ ]:
def detect_dye_edge(im,homography,lum_upp_lim=1,blue_low_lim=0.5,blue_red_prop=1,blue_green_prop=1,sigma=4,low_thresh=0.01,high_thresh=0.15):
    '''
    Pre-processing, Canny edge detection of blue dye streak, and morphological closing of edges.
    
    Parameters
    ----------
    im : 3D array
        Frame of video (in RGB color).
    homography : 2D array
        Homography mapping the original image to the perspective-warped image.
    lum_upp_lim : float, optional
        Luminance cutoff value; must be between 0 and 1. 
        All brighter pixels will not be considered as edge candidates.
    blue_low_lim : float, optional
        Blue channel lower cutoff value; must be between 0 and 1. 
        Pixels less blue will not be considered as edge candidates.
    blue_red_prop: float, optional
        Minimum ratio of blue to red value in a pixel required to be considered as an edge candidate.
    blue_green_prop: float, optional
        Minimum ratio of blue to green value in a pixel required to be considered as an edge candidate.
    sigma : float, optional
        Standard deviation of the Gaussian filter for Canny edge detection of channel boundaries.
    low_thresh : float, optional
        Lower bound for hysteresis thresholding (linking edges) for Canny edge detection.
    high_thresh : float, optional
        Upper bound for hysteresis thresholding (linking edges) for Canny edge detection.
        
    Returns
    -------
    edges: 2D array
        Mask of edges detected by Canny method.
    '''
    # Warp source image to destination based on homography
    im_warp = cv2.warpPerspective(im, homography, (im.shape[1],im.shape[0]))
    
    im_warp = im_warp/255.
    gray = rgb2gray(im_warp)
    out = np.zeros(im_warp.shape)

    # Mask based on luminance
    mask = gray < lum_upp_lim  # tune this value
    for i in range(3):
        out[mask, i] = np.copy(im_warp[mask, i])

    # Mask based on color
    imreds = np.copy(out[..., 0])
    imgreens = np.copy(out[..., 1])
    imblues = np.copy(out[..., 2])
    mask = (imblues >= blue_red_prop*imreds)&(imblues >= blue_green_prop*imgreens)&(imblues >= blue_low_lim) # tune this comparison
    out[~mask] = 0
    
    # Canny edge detection and morphological closing
    rough_edges = feature.canny(rgb2gray(out), sigma=sigma,low_threshold=low_thresh, high_threshold=high_thresh)  # tune sigma
    edges = morphology.binary_closing(rough_edges)
    
    return edges

In [ ]:
def get_ang_disp(edges,annular_params,rgarr,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid,plot=False,figuresize=(15,4),im=None):
    '''
    Parameters
    ----------
    edges: 2D array
        Mask of edges detected by Canny method.
    annular_params: list
        Annular domain parameters [cx_i,cy_i,r_u,r_l,r_i,r_o]:  (x) column-index of inner cylinder center, 
        (y) row-index of inner cylinder center, outer radius of annular domain, inner radius of annular domain
        inner radius of annular channel, outer radius of annular channel.
    rgarr : 1D array
        Array of radial gridpoints. dr = (r_u - r_l)/nr; rgarr = np.array([r_l + (1/2+n)*dr for n in range(nr)]).
    rmaskstack : 3D array
        Stack of circle masks for each radius in rgarr
    thetagrid : 2D array
        "True" angular position values on [0,2π). Theta=0 for x=0+ on the -y-axis and Theta=2π for x=0- on the -y-axis.
    theta2grid : 2D array
        Shifted angular position values on [-π,π). Theta=-π for x=0- on the +y-axis and Theta=π for x=0+ on the +y-axis.
    theta3grid : 2D array
        Shifted angular position values on [-π/2,3π/2). Theta=-π/2 for y=0- on the -x-axis and Theta=3π/2 for y=0+ on the -x-axis.
    theta4grid : 2D array
        Shifted angular position values on [π/2,5π/2). Theta=π/2 for y=0- on the +x-axis and Theta=5π/2 for y=0+ on the +x-axis.
    plot : boolean, optional
        If True, plots the detected leading edge and graph of angular displacment vs. radius
    figuresize : tuple, optional
        Dimensions of optional plot
    im : 3D array, optional
        RGB image. If plot=True, im must be provided.
        
    Returns
    -------
    angdisparr: 1D array
        Array of angular displacement values of the leading edge.
    '''
    # Mask the 2D array of angles with the edges
    thetaonedges = np.where(edges,thetagrid,np.nan)
    theta2onedges = np.where(edges,theta2grid,np.nan)
    theta3onedges = np.where(edges,theta3grid,np.nan)
    theta4onedges = np.where(edges,theta4grid,np.nan)

    # Mask
    thetaonedgesstack = rmaskstack*thetaonedges.reshape(thetaonedges.shape[0],thetaonedges.shape[1],1)
    theta2onedgesstack = rmaskstack*theta2onedges.reshape(theta2onedges.shape[0],theta2onedges.shape[1],1)
    theta3onedgesstack = rmaskstack*theta3onedges.reshape(theta3onedges.shape[0],theta3onedges.shape[1],1)
    theta4onedgesstack = rmaskstack*theta4onedges.reshape(theta4onedges.shape[0],theta4onedges.shape[1],1)

    maxtheta1arr= np.nanmax(thetaonedgesstack,axis=(0,1))
    maxtheta2arr= np.nanmax(theta2onedgesstack,axis=(0,1))
    maxtheta3arr= np.nanmax(theta3onedgesstack,axis=(0,1))
    maxtheta4arr= np.nanmax(theta4onedgesstack,axis=(0,1))

    maxthetastack = np.vstack((maxtheta1arr,maxtheta2arr,maxtheta3arr,maxtheta4arr))
    maxthetastack[maxthetastack<=0]=np.nan
    angdisparr = stats.mode(maxthetastack,nan_policy='omit')[0][0]
    
    if plot==True:
        [cx_o,cy_o,r_u,r_l,r_i,r_o] = annular_params
        
        im_and_edges = np.copy(im)
        for i in np.arange(im.shape[2]):
            im_and_edges[...,i]=im_and_edges[...,i]*(1-edges)
        im_and_edges[...,0]=im_and_edges[...,0]+edges
        
        fig, axd = plt.subplot_mosaic([['upper left', 'upper right'],['upper left', 'upper right'],
                               ['lower','lower']],figsize=figuresize)

        
        axd['upper left'].imshow(im_and_edges)
        axd['upper left'].scatter(cx_o+rgarr*np.sin(angdisparr),cy_o+rgarr*np.cos(angdisparr),color='#FF5F1F',s=1/60*figuresize[0]*figuresize[1])
        circ_u=plt.Circle((cx_o,cy_o),r_u,color='y',fill=False,linewidth=1.5)
        axd['upper left'].add_patch(circ_u)
        circ_l=plt.Circle((cx_o,cy_o),r_l,color='y',fill=False,linewidth=1.5)
        axd['upper left'].add_patch(circ_l)

        axd['upper right'].imshow(edges,cmap='binary')
        axd['upper right'].plot(cx_o+rgarr*np.sin(angdisparr),cy_o+rgarr*np.cos(angdisparr),color='#FF5F1F',linewidth=2/60*figuresize[0])
        axd['upper right'].scatter(cx_o+rgarr*np.sin(angdisparr),cy_o+rgarr*np.cos(angdisparr),color='#FF5F1F',s=1/60*figuresize[0]*figuresize[1])
        circ_u=plt.Circle((cx_o,cy_o),r_u,color='y',fill=False,linewidth=1.5)
        axd['upper right'].add_patch(circ_u)
        circ_l=plt.Circle((cx_o,cy_o),r_l,color='y',fill=False,linewidth=1.5)
        axd['upper right'].add_patch(circ_l)

        axd['lower'].plot(rgarr,angdisparr)
        axd['lower'].set_xlabel('$r$ (pixels)')
        axd['lower'].set_ylabel('$\\theta$')
        plt.show()
    
    return angdisparr

## Import video, metadata, and set sample rate for computing angular displacement ($\Delta\text{frames}$)

In [ ]:
fpaths = sorted(glob.glob(os.path.join('..','laboratory','laboratory-data','*.mp4')))
measurements = pd.read_csv(os.path.join('..','laboratory','laboratory-data','measurements.csv'))
nondim_exp_param = pd.read_csv(os.path.join('..','laboratory','laboratory-data','nondimensional-parameters.csv'))
cases = [re.search(r'.*Case-(.*).mp4',i).group(1).replace('_',' ') for i in fpaths]
runs = [nondim_exp_param[nondim_exp_param['Case']==case]['RUN'].to_numpy()[0] for case in cases]
print(cases)

In [ ]:
# CHOOSE CASE
case = 'I'
run = nondim_exp_param[nondim_exp_param['Case']==case]['RUN'].to_numpy()[0]
fname = os.path.join('..','laboratory','laboratory-data',f'Case-{case}.mp4')

In [ ]:
# Import data

run_index = nondim_exp_param[nondim_exp_param['RUN']==run].index.item()

In [ ]:
# Import video
if 'vdata' in globals():
    del vdata
vdata = skvideo.io.vread(fname)
metadata = skvideo.io.ffprobe(fname)
if measurements.at[run_index,'Flow direction']=='CW':
    vdata = np.flip(vdata,axis=2)

In [ ]:
# convert fraction strings to float (credit: https://stackoverflow.com/a/19073403)

def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        try:
            num, denom = frac_str.split('/')
        except ValueError:
            return None
        try:
            leading, num = num.split(' ')
        except ValueError:
            return float(num) / float(denom)        
        if float(leading) < 0:
            sign_mult = -1
        else:
            sign_mult = 1
        return float(leading) + sign_mult * (float(num) / float(denom))

In [ ]:
framerate = convert_to_float(metadata['video']['@avg_frame_rate']) # fps

$\Delta t_{min} = T_{circ}/\text{no. of pixels in mid-gap circle}$  
$\Delta t = 10 \Delta t_{min}$  
$\Delta \text{frames} = \text{frame rate} \times \Delta t$

In [ ]:
# Get experimental parameters
chi = nondim_exp_param.at[run_index,'R']
gamma = nondim_exp_param.at[run_index,'H']
U = nondim_exp_param.at[run_index,'U, MHD (m/s)']
ThreeTsp = nondim_exp_param.at[run_index,'3 tau (s)']
Tsp = 1/3*ThreeTsp
Tcirc = nondim_exp_param.at[run_index,'Tcirc (s)']
Tpeak = nondim_exp_param.at[run_index,'Tpeak (s)']
rpeak_r_o = nondim_exp_param.at[run_index,'rpeak/r_o']
r_i_meters = 1e-2*measurements[measurements['RUN']==run]['Inner radius (cm)'].item() # m
## Scaling from pixels to physical units of length (meters)
r_o_meters = 1e-2*measurements[measurements['RUN']==run]['Outer radius (cm)'].item() # m

#### Implement 2D analytical solution:

In [ ]:
from scipy.special import i1,k1
def An(r,n,H,R):
    an = (16*(1 + R)*(i1(((-0.5 + n)*np.pi*r)/H)*(-(r*k1(((-0.5 + n)*np.pi)/H)) + R*r*k1((R*(-0.5 + n)*np.pi)/H)) + 
            R*i1((R*(-0.5 + n)*np.pi)/H)*(k1(((-0.5 + n)*np.pi)/H) - r*k1(((-0.5 + n)*np.pi*r)/H)) + 
            i1(((-0.5 + n)*np.pi)/H)*(-(R*k1((R*(-0.5 + n)*np.pi)/H)) + 
            r*k1(((-0.5 + n)*np.pi*r)/H))))/(R*(-1 + 2*n)**3*np.pi**3*r*(i1((R*(-0.5 + n)*np.pi)/H)*k1(((-0.5 + n)*np.pi)/H) - 
            i1(((-0.5 + n)*np.pi)/H)*k1((R*(-0.5 + n)*np.pi)/H)))
    return an
        
def u2D(r,z,l,H,R):
    u2Darr = np.array([An(r,n,H,R)*np.sin((-0.5 + n)*np.pi*z) for n in range(1,l+1)])
    return u2Darr.sum(axis=0)

## Analysis

#### Precomputation and tuning detection parameters

In [ ]:
# SET PARAMETERS (see 'dye-tracking_parameters.csv' for the parameters used in David et al. (2024))

# Manually adjust center if necessary
nr = 15
dxi = 0
dyi = 0
dxo = 10
dyo = -4
expand = 1.05
contract = 0.97
α=1.15
m=2.5
houghcannysigma = 3
houghcannylowthresh=10
houghcannyhighthresh=35

# Tune edge detection parameters
lum_upp_lim=1
blue_low_lim=0.5
blue_red_prop=1
blue_green_prop=1
sigma=3
low_thresh=0.01
high_thresh=0.15

# File-saving settings
use_preexisting_params = True
overwrite_preexisting_params = False

# Make or import data file of tunable parameters
datapath = 'data'

param_keys = ['RUN', 'Luminance upper lim', 'Blue lower lim','Blue to red','Blue to green',
            'Canny sigma','Canny lower thresh','Canny upper thresh', 
            'Inner radius expand','Outer radius contract','houghcannysigma',
            'houghcannylowthresh','houghcannyhighthresh','dxi','dyi','dxo','dyo','grid alpha','grid m','nr']
param_values = [run,lum_upp_lim,blue_low_lim,blue_red_prop,blue_green_prop,sigma,low_thresh,high_thresh,
         expand,contract,houghcannysigma,houghcannylowthresh,houghcannyhighthresh,dxi,dyi,dxo,dyo,α,m,nr]

if not os.path.exists(datapath):
    os.makedirs(datapath)
if not os.path.exists(datapath+'/tunable_parameters.csv'):
    tunableparams = pd.DataFrame(columns = param_keys)
    tunableparams.to_csv(datapath+'/tunable_parameters.csv',index=False)

tunableparams = pd.read_csv(datapath+'/tunable_parameters.csv')

if np.sum((tunableparams['RUN']==run).values)==0:
    # append new row
    tunableparams = pd.concat([tunableparams,pd.DataFrame(dict(zip(param_keys, param_values)),index=[0])],ignore_index = True)
else:
    run_index_tp = tunableparams[tunableparams['RUN']==run].index.item()
    
    # Use pre-existing values?
    if use_preexisting_params == True:
        [run,lum_upp_lim,blue_low_lim,blue_red_prop,blue_green_prop,sigma,low_thresh,high_thresh,
         expand,contract,houghcannysigma,houghcannylowthresh,houghcannyhighthresh,dxi,dyi,dxo,dyo,α,m,nr] = tunableparams.loc[run_index_tp,:]
    
    else:
        if overwrite_preexisting_params == True:
            # replace existing row
            tunableparams.loc[run_index_tp,:] = param_values

tunableparams.to_csv(datapath+'/tunable_parameters.csv',index=False)

In [ ]:
homography,annular_params,scale_factor,rgarr,rmg_index,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid = make_polar_meshgrids(
    vdata,nr=nr,r_o_meters=r_o_meters,warp=False,α=α,m=m,chi_channel=chi,plot=True,cannysigma=houghcannysigma,expand=expand,contract=contract,
    cannylowthresh=houghcannylowthresh,cannyhighthresh=houghcannyhighthresh,dcxi=dxi,dcyi=dyi,dcxo=dxo,dcyo=dyo,figuresize=(10,10),plotzoom=1)

In [ ]:
tuneim=vdata[int(ThreeTsp*framerate)]

tuneedges_lead = detect_dye_edge(tuneim,homography,lum_upp_lim,blue_low_lim,blue_red_prop,blue_green_prop,sigma,low_thresh,high_thresh)
tuneangdisparr_lead = get_ang_disp(tuneedges_lead,annular_params,rgarr,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid,plot=True,figuresize=(20,15),im=cv2.warpPerspective(tuneim, homography, (tuneim.shape[1],tuneim.shape[0])))
tuneedges_trail = detect_dye_edge(tuneim,homography,lum_upp_lim,blue_low_lim,blue_red_prop,1.1*blue_green_prop,sigma,low_thresh,high_thresh)
tuneangdisparr_trail = get_ang_disp(tuneedges_trail,annular_params,rgarr,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid,plot=True,figuresize=(20,15),im=cv2.warpPerspective(tuneim, homography, (tuneim.shape[1],tuneim.shape[0])))

#### Set temporal resolution

In [ ]:
plt.plot(scale_factor*rgarr/r_o_meters,tuneangdisparr_lead,label='tuned edge')
plt.plot(scale_factor*rgarr/r_o_meters,tuneangdisparr_trail,label='blue:green +10%')
plt.legend(frameon=False)
plt.xlabel('$\\tilde{r}$')
plt.ylabel('$\\theta$')

In [ ]:
plt.plot(scale_factor*rgarr/r_o_meters,tuneangdisparr_lead-tuneangdisparr_trail)
plt.xlabel('$\\tilde{r}$')
plt.ylabel('$\\sigma_\\theta$')

In [ ]:
# Compute time-step based on uncertainty in dye streak position (sigmatheta)
angdyewidth = (tuneangdisparr_lead -(2*np.pi-tuneangdisparr_trail)).data
sigmatheta = (tuneangdisparr_lead -tuneangdisparr_trail).data#angdyewidth/4
u_dim_pred = U*u2D(scale_factor*rgarr/r_o_meters,1,3,gamma,chi)
dt_sigmatheta_arr = scale_factor*rgarr/u_dim_pred * sigmatheta
dt_sigmatheta = np.percentile(dt_sigmatheta_arr,80)
# Compute minimum pixel-crossing time
num_pix_at_each_r = np.array([np.sum(rmaskstack[...,i]) for i in np.arange(rmaskstack.shape[2])])
dt_pix_arr = scale_factor*rgarr/u_dim_pred * 1/num_pix_at_each_r
dt_pix = np.max(dt_pix_arr)

# Choose the timestep based on the following criterion
dt_est = np.max([np.max([10*dt_pix,3*dt_sigmatheta]),Tsp/2])

if dt_est>Tsp/2:
    print('Time step too large')
else:

    # File-saving settings
    use_preexisting_timestep = False
    overwrite_preexisting_timestep = True

    # Make or import data file of tunable parameters
    timestep_keys = ['RUN', 'Tsp', 'dt_pix','dt_sigmatheta','dt_est']
    timestep_values = [run, Tsp, dt_pix, dt_sigmatheta, dt_est]

    if not os.path.exists(datapath+'/timestep.csv'):
        timesteps = pd.DataFrame(columns = timestep_keys)
        timesteps.to_csv(datapath+'/timestep.csv',index=False)

    timesteps = pd.read_csv(datapath+'/timestep.csv')

    if np.sum((timesteps['RUN']==run).values)==0:
        # append new row
        timesteps = pd.concat([timesteps,pd.DataFrame(dict(zip(timestep_keys, timestep_values)),index=[0])],ignore_index = True)
    else:
        run_index_ts = timesteps[timesteps['RUN']==run].index.item()

        # Use pre-existing values?
        if use_preexisting_timestep == True:
            [_,_,_,_, dt_est] = timesteps.loc[run_index_ts,:]

        else:
            if overwrite_preexisting_timestep == True:
                # replace existing row
                timesteps.loc[run_index_ts,:] = timestep_values

    timesteps.to_csv(datapath+'/timestep.csv',index=False)

    # Compute frame step
    dframes = int(np.round(framerate*dt_est))

    # Trim times beyond Tcirc/2 + dt_est
    trim_frame = int(np.round(Tcirc/2*framerate)+dframes)
    if trim_frame < vdata.shape[0]:
        unshifted_framearr = np.arange(0,trim_frame+1,dframes)
    else:
        unshifted_framearr = np.arange(0,vdata.shape[0],dframes)

    # Shift frame array to minimize distance of nearest time to 3 Tsp
    three_tau_frame = int(np.round(ThreeTsp*framerate))
    frameshift = (three_tau_frame - unshifted_framearr)[np.argmin(np.abs(three_tau_frame - unshifted_framearr))]
    framearr = (unshifted_framearr + frameshift)[(unshifted_framearr + frameshift)>=0]
    tarr = framearr/framerate

    # Get indices of tarr and framearr that are closest to multiples of Tsp
    num_Tsp = int(np.round(tarr[-1]/Tsp))
    Tsp_indices = np.zeros(num_Tsp)
    for n in np.arange(num_Tsp):
        Tsp_indices[n] = int(np.argmin(np.abs(tarr-(n+1)*Tsp)))
    Tsp_indices = Tsp_indices.astype(int)

In [ ]:
np.arange(0,vdata.shape[0]+1,dframes)

In [ ]:
pd.DataFrame({'RUN':timesteps['RUN'],'3 dt_sigmatheta/Tsp':3*timesteps['dt_sigmatheta']/timesteps['Tsp'],'dt_est/Tsp':timesteps['dt_est']/timesteps['Tsp']})

#### Main loop

In [ ]:
%%time

angdispstack = np.zeros((len(rgarr),len(framearr)))
altangdispstack = np.zeros((len(rgarr),len(framearr)))

for i in np.arange(len(framearr)):
    frame = framearr[i]
    im = vdata[frame]
    edges = detect_dye_edge(im,homography,lum_upp_lim,blue_low_lim,blue_red_prop,blue_green_prop,sigma,low_thresh,high_thresh)
    altedges = detect_dye_edge(im,homography,lum_upp_lim,blue_low_lim,blue_red_prop,1.1*blue_green_prop,sigma,low_thresh,high_thresh)
    angdisparr = get_ang_disp(edges,annular_params,rgarr,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid)
    angdispstack[:,i] = angdisparr
    altangdisparr = get_ang_disp(altedges,annular_params,rgarr,rmaskstack,thetagrid,theta2grid,theta3grid,theta4grid)
    altangdispstack[:,i] = altangdisparr

In [ ]:
radiusarr = scale_factor*rgarr

#### Corrections

In [ ]:
# Correct angular displacement discontinuities that arise from branch-cuts
correctedangdispstack = np.zeros(angdispstack.shape)
altcorrectedangdispstack = np.zeros(altangdispstack.shape)

for rindex in np.arange(len(rgarr)):
    r = rgarr[rindex]
    angtimeseries = angdispstack[rindex,:]
    altangtimeseries = altangdispstack[rindex,:]
    correctedangtimeseries = np.copy(angtimeseries)
    altcorrectedangtimeseries = np.copy(altangtimeseries)
    angdiffs = np.diff(correctedangtimeseries)
    altangdiffs = np.diff(altcorrectedangtimeseries)
    while np.min(angdiffs)<=-0.6*2*np.pi:
        for angindex in np.arange(len(angtimeseries)-1):
            if angdiffs[angindex]<=-0.6*2*np.pi:
                correctedangtimeseries[angindex+1:]=correctedangtimeseries[angindex+1:]+2*np.pi
        angdiffs = np.diff(correctedangtimeseries)
    while np.min(altangdiffs)<=-0.6*2*np.pi:
        for angindex in np.arange(len(altangtimeseries)-1):
            if altangdiffs[angindex]<=-0.6*2*np.pi:
                altcorrectedangtimeseries[angindex+1:]=altcorrectedangtimeseries[angindex+1:]+2*np.pi
        altangdiffs = np.diff(altcorrectedangtimeseries)
        
    correctedangdispstack[rindex,:]=correctedangtimeseries
    altcorrectedangdispstack[rindex,:]=altcorrectedangtimeseries

#### Outlier detection

In [ ]:
filledangdispstack = np.copy(correctedangdispstack)
altfilledangdispstack = np.copy(altcorrectedangdispstack)
transformer = HampelFilter(window_length=5)
for tindex in np.arange(len(tarr)):
    angprof = correctedangdispstack[:,tindex]
    altangprof = altcorrectedangdispstack[:,tindex]
    angprof_hat = np.array([i[0] for i in transformer.fit_transform(angprof)])
    altangprof_hat = np.array([i[0] for i in transformer.fit_transform(altangprof)])
    transformer_imputer = Imputer(method="linear")
    angprof_hat_filled =  np.array([i[0] for i in transformer_imputer.fit_transform(angprof_hat)])
    altangprof_hat_filled =  np.array([i[0] for i in transformer_imputer.fit_transform(altangprof_hat)])
    filledangdispstack[:,tindex] = angprof_hat_filled
    altfilledangdispstack[:,tindex] = altangprof_hat_filled

#### Uncertainty in angular displacement

In [ ]:
sigmaangdispstack = np.abs(filledangdispstack-altfilledangdispstack)

#### Estimating velocity and associated uncertainty

In [ ]:
%%time
# Take finite difference with respect to time
angvelstack =  np.gradient(filledangdispstack,tarr,axis=1) # d_dt(filledangdispstack) 
velstackms = radiusarr[:, None]*angvelstack

# Estimate uncertainty using Monte Carlo method
it = 10000
angdispsamples = default_rng().normal(loc=filledangdispstack,scale=sigmaangdispstack,size=(it,*np.shape(filledangdispstack)))
angvelsamples =  np.gradient(angdispsamples,tarr,axis=2)
velsamples = radiusarr[None,:, None]*angvelsamples
velsamplesmean = np.mean(velsamples,axis=0)
sigmavelstackms= np.std(velsamples,axis=0)

#### Save data

In [ ]:
datafolderpath = os.path.join(datapath,run)
if not os.path.exists(datafolderpath):
    os.makedirs(datafolderpath)

np.savetxt(os.path.join(datafolderpath,'correctedangdisps-'+run+'.csv'), correctedangdispstack, delimiter=',')
np.savetxt(os.path.join(datafolderpath,'radiusarr-'+run+'.csv'), radiusarr, delimiter=',')
np.savetxt(os.path.join(datafolderpath,'timearr-'+run+'.csv'), tarr, delimiter=',')
np.savetxt(os.path.join(datafolderpath,'Tspindices-'+run+'.csv'), Tsp_indices, delimiter=',')
np.savetxt(os.path.join(datafolderpath,'rmgindex-'+run+'.csv'), [rmg_index], delimiter=',',fmt='%s')
np.savetxt(os.path.join(datafolderpath,'velocities-'+run+'.csv'), velstackms, delimiter=',')
np.savetxt(os.path.join(datafolderpath,'uncertainties-velocities-'+run+'.csv'), sigmavelstackms, delimiter=',')

## Plotting

In [ ]:
for t in np.flip(np.arange(np.shape(velsamples)[2])):
    jitter = np.random.normal(loc=0,scale=0.0002)
    p=plt.plot(radiusarr/r_o_meters,velsamplesmean[:,t]/U,label="t/τsp = "+str(np.round(tarr[t]/(ThreeTsp/3),1)))
    plt.scatter(radiusarr/r_o_meters,velsamples[0,:,t]/U,color=p[0].get_color(),s=1,alpha=0.2)
    #plt.errorbar(radiusarr/r_o_meters,velstackms[:,t]/U,sigmavelstackms[:,t]/U,
                   #color=p[0].get_color(),ls='none',capthick=0.7,elinewidth=0.7,capsize=1.5,alpha=1)
    for s in np.arange(1,np.shape(velsamples)[0],int(np.shape(velsamples)[0]/50)):
        plt.scatter((radiusarr+jitter)/r_o_meters,velsamples[s,:,t]/U,color=p[0].get_color(),s=10,alpha=0.05)
plt.ylim(0,3)
plt.legend(bbox_to_anchor=(1.2, 0.5), loc="center",frameon=False)
plt.xlim(chi,1)
plt.ylim(0,1.2*(1+chi)/2*1/chi)
plt.xlabel('$\\tilde{r}$')
plt.ylabel('$\\tilde{u}_\\theta$')

In [ ]:
timeindex=-1#int(avgendindex/1)-8
radindex=rmg_index

radsmoothcolor = '#FFB48E'#FFC08E'
timesmoothcolor = '#F67BB0'#'#FCAE03'#'#FF76B8'#'#FE5A9D''#FF5252'
avgcolor = '#008FB2'#'#0086BC'#'#00932A'
rtheorarr = np.linspace(chi,1,100)
ttheorarr = np.linspace(0,tarr[-1],100)

fig, axd = plt.subplot_mosaic([['left', 'upper right'],
                               ['left', 'lower right']],
                              figsize=(15, 7), constrained_layout=False)
axd['left'].imshow(cv2.warpPerspective(vdata[framearr[timeindex],...], homography, (vdata[framearr[timeindex],...].shape[1],vdata[framearr[timeindex],...].shape[0])))
axd['left'].plot(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex]),color='gray')
axd['left'].scatter(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex]),color='#FF821F',s=12)
rcircpatch = plt.Circle((annular_params[0],annular_params[1]),rgarr[radindex],color='k',fill=False,linewidth=1.5,linestyle='--')
axd['left'].add_patch(rcircpatch)
axd['left'].set_title('Dye streak at $t=$'+str(np.round(tarr[timeindex]))+'s with\nraw and radialy-smoothed Canny edges')

p=axd['upper right'].plot(ttheorarr/Tsp,100*U*(1-np.exp(-ttheorarr/Tsp))*u2D(radiusarr[radindex]/r_o_meters,1,3,gamma,chi),alpha=0.5)
axd['upper right'].scatter(tarr/Tsp,100*velstackms[radindex,:],color=p[0].get_color())
axd['upper right'].errorbar(tarr/Tsp,100*velstackms[radindex,:],100*sigmavelstackms[radindex,:],
                   ls='none',capthick=0.7,elinewidth=0.7,capsize=1.5,alpha=1,color=p[0].get_color())
axd['upper right'].scatter(tarr[timeindex]/Tsp,0,marker='d',color='k',s=50)
axd['upper right'].set_ylim(0,1.5*np.max(100*velstackms[radindex,:]))
axd['upper right'].set_xlim(0,)
#axd['upper right'].legend(bbox_to_anchor=(1.05,1))
axd['upper right'].set_title('Azimuthal velocity at $r=$'+str(np.round(100*radiusarr[radindex]))+'cm over time')
axd['upper right'].set_xlabel('$t/\\tau_{sp}$')
axd['upper right'].set_ylabel('$u_{\\theta}$ (cm/s)')

axd['lower right'].scatter(100*radiusarr,100*velstackms[:,timeindex])
axd['lower right'].plot(100*r_o_meters*rtheorarr,100*U*(1-np.exp(-tarr[timeindex]/Tsp))*u2D(rtheorarr,1,3,gamma,chi),alpha=0.5)
axd['lower right'].scatter(100*radiusarr[radindex],0,marker='d',color='k',s=50)
axd['lower right'].errorbar(100*radiusarr,100*velstackms[:,timeindex],100*sigmavelstackms[:,timeindex],
                   ls='none',capthick=0.7,elinewidth=0.7,capsize=1.5,alpha=1,color=p[0].get_color())
axd['lower right'].set_ylim(0,100*1.3*np.quantile(velstackms,q=0.95))
axd['lower right'].set_xlim(r_i_meters*100,r_o_meters*100)
axd['lower right'].set_title('Radial profile of azimuthal velocity at $t=$'+str(np.round(tarr[timeindex]))+'s')
axd['lower right'].set_ylabel('$u_{\\theta}$ (cm/s)')
axd['lower right'].set_xlabel('$r$ (cm)')

plt.tight_layout()

In [ ]:
timeindex2=Tsp_indices[2]
timeindex1=timeindex2-1

fig, axd = plt.subplot_mosaic([['left', 'right']],
                              figsize=(15, 7), constrained_layout=False)
axd['left'].imshow(cv2.warpPerspective(vdata[framearr[timeindex1],...], homography, (vdata[framearr[timeindex1],...].shape[1],vdata[framearr[timeindex1],...].shape[0])))
axd['left'].plot(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex1]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex1]),color='gray',linewidth=1)
axd['left'].scatter(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex1]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex1]),color='#FF821F',s=8)
rcircpatch = plt.Circle((annular_params[0],annular_params[1]),rgarr[radindex],color='k',fill=False,linewidth=1.5,linestyle='--')
axd['left'].add_patch(rcircpatch)
axd['left'].set_title('Dye streak at $t=$'+str(np.round(tarr[timeindex1],2))+'s with\nraw and radialy-smoothed Canny edges')

axd['right'].imshow(cv2.warpPerspective(vdata[framearr[timeindex2],...], homography, (vdata[framearr[timeindex2],...].shape[1],vdata[framearr[timeindex2],...].shape[0])))
axd['right'].plot(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex2]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex2]),color='gray',linewidth=1)
axd['right'].scatter(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex2]),
         annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex2]),color='#FF821F',s=8)
rcircpatch = plt.Circle((annular_params[0],annular_params[1]),rgarr[radindex],color='k',fill=False,linewidth=1.5,linestyle='--')
axd['right'].add_patch(rcircpatch)
axd['right'].set_title('Dye streak at $t=$'+str(np.round(tarr[timeindex1],2))+'s with\nraw and radialy-smoothed Canny edges')

In [ ]:
im1=np.zeros(np.shape(vdata[framearr[timeindex1],...]))
im2=np.zeros(np.shape(vdata[framearr[timeindex2],...]))
im1[...,0] = rgb2gray(cv2.warpPerspective(vdata[framearr[timeindex1],...], homography, (vdata[framearr[timeindex1],...].shape[1],vdata[framearr[timeindex1],...].shape[0])))
im1[...,2] = im1[...,0]
im2[...,1] = rgb2gray(cv2.warpPerspective(vdata[framearr[timeindex2],...], homography, (vdata[framearr[timeindex2],...].shape[1],vdata[framearr[timeindex2],...].shape[0])))

plt.figure(figsize=(5,5))
plt.imshow(im1+im2)
plt.gca().axes.get_xaxis().set_ticks([])
plt.gca().axes.get_yaxis().set_ticks([])

#plt.plot(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex1]),
         #annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex1]),color='gray')
plt.scatter(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex1]),
            annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex1]),
            s=12,facecolor='limegreen',edgecolor='k',linewidth=0.2,zorder=2,
            label='$t/\\tau_{sp}$ = '+'{t_tsp1:.1f}'.format(t_tsp1=tarr[timeindex1]/Tsp))
plt.scatter(annular_params[0]+rgarr*np.sin(filledangdispstack[:,timeindex2]),
            annular_params[1]+rgarr*np.cos(filledangdispstack[:,timeindex2]),
            s=12,facecolor='fuchsia',edgecolor='k',linewidth=0.2,zorder=2,
            label='$t/\\tau_{sp}$ = '+'{t_tsp2:.1f}'.format(t_tsp2=tarr[timeindex2]/Tsp))

for i in np.arange(len(filledangdispstack[:,timeindex1])):
    x1 = annular_params[0]+rgarr[i]*np.sin(filledangdispstack[i,timeindex1])
    y1 = annular_params[1]+rgarr[i]*np.cos(filledangdispstack[i,timeindex1])
    x2 = annular_params[0]+rgarr[i]*np.sin(filledangdispstack[i,timeindex2])
    y2 = annular_params[1]+rgarr[i]*np.cos(filledangdispstack[i,timeindex2])
    
    arrow_dr = 3
    r = np.sqrt((x2-x1)**2+(y2-y1)**2)
    radjusted = r-arrow_dr
    
    arrow_dx = radjusted*np.cos(np.arctan2((y2-y1),(x2-x1)))
    arrow_dy = radjusted*np.sin(np.arctan2((y2-y1),(x2-x1)))
    plt.arrow(x1,y1,arrow_dx,arrow_dy,linewidth=0.5,head_width=10,head_length=10,length_includes_head=True,
              facecolor='k',zorder=1)
plt.title('Displacement of dye over one\ntime step (from $t/\\tau_{sp}$ = '+'{t_tsp1:.1f} to {t_tsp2:.1f})'.format(t_tsp1=tarr[timeindex1]/Tsp,t_tsp2=tarr[timeindex2]/Tsp)+' for '+run)
leg = plt.legend(bbox_to_anchor=(0.04, 0.13), bbox_transform=fig.transFigure, markerscale=3,loc="lower left",ncol=1,labelspacing=1,frameon=True,framealpha=1)    
leg.get_frame().set_color('white')
leg.get_frame().set_linewidth(4)
plt.tight_layout()

In [ ]:
for ti in np.flip(np.arange(0,len(tarr),1)):#Tsp_indices:
    p=plt.plot(100*radiusarr,filledangdispstack[:,ti]/np.pi,label="t/τsp = "+str(np.round(tarr[ti]/(ThreeTsp/3),1)))

    plt.errorbar(100*radiusarr,filledangdispstack[:,ti]/np.pi,sigmaangdispstack[:,ti]/np.pi,ls='none',
                 color=p[-1].get_color(),capthick=0.7,elinewidth=0.7,capsize=1.5)
    
    #plt.plot(100*radiusarr,altfilledangdispstack[:,ti]/np.pi,linestyle='--',color=p[-1].get_color())
    
plt.xlabel("r (cm)")
plt.ylabel("θ/π")
plt.legend(bbox_to_anchor=(1.2, 0.5), loc="center",frameon=False)
plt.ylim(0,)
plt.tight_layout()
plt.show()

In [ ]:
plotnum = 3

rtheorarr = np.linspace(chi,1,100)
fig, ax = plt.subplots(1,plotnum,figsize=(5*plotnum,3))

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i in np.arange(plotnum):
    ax[i].scatter(radiusarr/r_o_meters,velstackms[:,Tsp_indices[i]]/U,facecolor=colors[0],edgecolor=colors[0],s=20,alpha=1,zorder=3)
    ax[i].errorbar(radiusarr/r_o_meters,velstackms[:,Tsp_indices[i]]/U,sigmavelstackms[:,Tsp_indices[i]]/U,
                   color=colors[0],ls='none',capthick=0.7,elinewidth=0.7,capsize=1.5,alpha=1)
    ax[i].plot(rtheorarr,(1-np.exp(-tarr[Tsp_indices[i]]/Tsp))*u2D(rtheorarr,1,3,gamma,chi),alpha=0.5)
    ax[i].set_xlim(chi,1)
    ax[i].set_ylim(0,1.*(1+chi)/2*1/chi)
    ax[i].set_title('$t/\\tau_{sp}$ = '+'{t:.1f}'.format(t=tarr[Tsp_indices[i]]/Tsp))
    ax[i].set_xlabel('$\\tilde{r}$')
    ax[i].set_ylabel('$\\tilde{u}_\\theta$')